In [1]:
import pandas as pd
import ccxt
from binance.client import Client
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib
from datetime import datetime, timedelta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import SMA


In [2]:
API_KEY = 'O8X3LMqT928iS3jAV1EwM2uwSedUkWEigt2TELCUJUfdhKT3zWVgxExAWCJrUKe5'
API_SECRET = 'eNQIdMOA9FieSiQh1jK2JSwK9tNERiSpVNRLQoKC6a1vqqmwG2JNMgbUs69O4IhE'
client = Client(API_KEY,API_SECRET)

In [3]:
#Creating DataFrame using Binance, data of BTC/USDT

# exchange = ccxt.binance()
symbol = 'ETH/USDT'
timeframe = '4h'
ohlcv = client.get_historical_klines('ETHUSDT',Client.KLINE_INTERVAL_4HOUR,'16 Ago, 2017','06 Mar, 2022')
to_df = pd.DataFrame(ohlcv)
clean_df = to_df.drop(columns=[0,6,7,8,9,10,11])
clean_df.columns = ['Open','High','Low','Close','Volume']
clean_df['Open'] = pd.to_numeric(clean_df['Open'])
clean_df['High'] = pd.to_numeric(clean_df['High'])
clean_df['Low'] = pd.to_numeric(clean_df['Low'])
clean_df['Close'] = pd.to_numeric(clean_df['Close'])
clean_df['Volume'] = pd.to_numeric(clean_df['Volume'])


In [6]:
open = clean_df['Open']
high = clean_df['High']
low = clean_df['Low']
close = clean_df['Close']


candle_names = talib.get_function_groups()['Pattern Recognition']
candles = [i for i in candle_names if i == 'CDLENGULFING' or i == 'CDLHAMMER' or i == 'CDLEVENINGSTAR' ]

for candle in candles:
    clean_df[candle] = getattr(talib, candle) (open,high,low,close)

print(clean_df['Close'].iloc[0])

307.96


In [17]:
class SMACross(Strategy):
    mm1 = 8
    mm2 = 20
    mm3 = 30
    mm4 = 200

    def init(self):
        super().init()
        self.sma8 = self.I(SMA, self.data.Close, self.mm1)
        self.sma20 = self.I(SMA, self.data.Close, self.mm2)
        self.sma30 = self.I(SMA, self.data.Close, self.mm3)
        self.sma200 = self.I(SMA, self.data.Close, self.mm4)

    def next(self):
        super().next()
        self.stop_loss = 0.10
        self.take_profit = 0.20
        self.size = 10000 * 0.10
        self.price = clean_df['Close'].iloc[-1]
        self.engulfing = talib.CDLENGULFING(clean_df['Open'], clean_df['High'], clean_df['Low'], clean_df['Close'])
        # self.engulfing_bullish = talib.CDLENGULFING(clean_df['Open'], clean_df['Low'], clean_df['High'], clean_df['Close'])
        
        if not self.position:
            if self.sma20 > self.sma30 and self.price > self.sma30:
                self.buy()
            elif self.sma20 < self.sma30 and self.price < self.sma30:
                self.sell()
        else:
            if self.position.is_long and self.sma8 < self.sma20 or \
            self.position.is_short and self.sma8 > self.sma20:
                self.position.close()


bt = Backtest(clean_df,SMACross, cash = 10000, commission = 0.00, trade_on_close=False)
output = bt.run()
print(output)
bt.plot()


C:\Users\MI PC\AppData\Local\Temp\ipykernel_7128\748020679.py:34: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(clean_df,SMACross, cash = 10000, commission = 0.00, trade_on_close=False)


Start                                     0.0
End                                    9954.0
Duration                               9954.0
Exposure Time [%]                   54.736313
Equity Final [$]                    191801.14
Equity Peak [$]                     390158.59
Return [%]                          1818.0114
Buy & Hold Return [%]              757.013898
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -57.424489
Avg. Drawdown [%]                   -8.580335
Max. Drawdown Duration                 2302.0
Avg. Drawdown Duration             112.267442
# Trades                                840.0
Win Rate [%]                        46.547619
Best Trade [%]                      66.794315
Worst Trade [%]                    -15.739238
Avg. Trade [%]                    

Row(id='9923', ...)